# Basic CNN Classifier

Basic CNN Classifier에서는 비교적 적은 층을 가진 CNN 모델을 만들어봅니다. 데이터셋은 MNIST를 사용할 것이며 추가적으로 Image Preprocessing 과정에 대해 배웁니다.

`torchvision`에는 `torch`에서 CV에 필요한 클래스들이 선언되어 있습니다. 그중 `torchvision.transoforms`에는 Image Preprocessing 과정에 필요한 많은 메서드들이 들어있습니다.

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MNIST는 28x28x1의 사이즈를 가진 이미지 데이터셋입니다. 즉 Gray Scale 이미지이므로 `channel_size`를 1로 선언하였습니다. 또한 가로/세로 크기가 같은 정사각형 형태의 이미지이므로 `image_size`를 28로 선언하였습니다.

In [3]:
image_size = 28
channel_size = 1
epochs = 5
batch_size = 128
learning_rate = 0.001
nb_classes = 10

`transoforms.Compose`는 여러개의 변형을 한번에 적용시킵니다. 여러가지 `torchvision.transoforms`에는 여러가지 Augmentation 기법들 또한 포함되어 있습니다. 주의해야할 점은 test 데이터셋에 사용할 변환에는 Augmentation 기법을 사용하지 않습니다.

In [4]:
train_T = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

test_T = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

`dataset`의 `transform` 매개변수에는 변환이 인자로 올 수 있습니다.

In [5]:
train_dataset = torchvision.datasets.MNIST(root='./MNIST',
                                           train=True,
                                           transform=train_T,
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root="./MNIST",
                                          train=False,
                                          transform=test_T)

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

`nn.Sequential()`을 사용하면 네트워크를 한번에 쌓을 수 있습니다.
FC 부분에서는 3-Dimention을 가진 Feature Map을 1차원으로 길게 펴줘야합니다.

In [7]:
class Model(nn.Module):
    def __init__(self, nb_classes):
        super(Model, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channel_size, 64, kernel_size=3, stride=1, padding=2),
            # Out Feature Map Size : 30*30*64
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # Out Feature Map Size : 15*15*64
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=2),
            # Out Feature Map Size : 17*17*128
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            # Out Feature Map Size : 8*8*128
        )
        self.fc = nn.Sequential(
            nn.Linear(8*8*128, 128),
            nn.Dropout2d(0.5),
            nn.Linear(128, nb_classes)
        )
    
    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1) # Flaaten 과정
        out = self.fc(out)
        
        return out

In [8]:
model = Model(nb_classes).to(device)

In [9]:
cost_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_step = len(train_loader)

In [11]:
for epoch in range(epochs):
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = cost_function(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if not (idx+1) % 100:
            print ('Epoch [{}/{}], Step [{}/{}], Loss={}'
                       .format(epoch+1, epochs, idx+1, total_step, loss.item()))

Epoch [1/5], Step [100/469], Loss=0.11442941427230835
Epoch [1/5], Step [200/469], Loss=0.11423785239458084
Epoch [1/5], Step [300/469], Loss=0.1263194978237152
Epoch [1/5], Step [400/469], Loss=0.0738304853439331
Epoch [2/5], Step [100/469], Loss=0.070501908659935
Epoch [2/5], Step [200/469], Loss=0.08052519708871841
Epoch [2/5], Step [300/469], Loss=0.044242870062589645
Epoch [2/5], Step [400/469], Loss=0.08847593516111374
Epoch [3/5], Step [100/469], Loss=0.04784110188484192
Epoch [3/5], Step [200/469], Loss=0.035116732120513916
Epoch [3/5], Step [300/469], Loss=0.04577498883008957
Epoch [3/5], Step [400/469], Loss=0.03421831503510475
Epoch [4/5], Step [100/469], Loss=0.06962677836418152
Epoch [4/5], Step [200/469], Loss=0.03003201261162758
Epoch [4/5], Step [300/469], Loss=0.036226868629455566
Epoch [4/5], Step [400/469], Loss=0.02325938269495964
Epoch [5/5], Step [100/469], Loss=0.056615039706230164
Epoch [5/5], Step [200/469], Loss=0.02293083444237709
Epoch [5/5], Step [300/469],

`model.eval()`은 테스트를 진행할 때 수행해주어야 하는 구문입니다. Batch Normalize 혹은 Dropout과 같은 기법들은 학습 단계에서만 활성화 되어야 합니다. `model.eval()`은 이러한 학습 단계에서만 활성화 되어야 하는 기법들을 비활성화 시킵니다.

In [12]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print("Accuracy: {}".format(100*correct / total))

Accuracy: 98.93
